In [1]:
import time
import cv2
import math

import numpy as np

from swarm import Swarm


In [2]:
NUM_DRONES = 5

AREA_SIZE_X = 100
AREA_SIZE_Y = 100
GRID_SIZE = 100           # Size of the Forest

CAMERA_FOV_DEGREE = 50
IMAGE_SIZE = 512 #How many Scanning points each image has per Row (Images are 512x512)

MOVE_DISTANCE = 1       # How far a Drone can move each Timestep
DRONE_HEIGHT = 35

TIMESTEPS = 10
POPULATION_SIZE = 10
NUM_GENERATIONS = 1
MUTATION_RATE = 0.1

INITIAL_VISIBILITY =0.5

In [3]:
grid_offset=GRID_SIZE/2

camera_fov=np.deg2rad(CAMERA_FOV_DEGREE)
camera_offset= 35 * np.tan(camera_fov/2)

prob_array_size= int(np.ceil((GRID_SIZE*512)/(camera_offset*2)))

In [4]:
#Create "Base Grid"
#by Calculating distance between each scanning point at DRONE_HEIGHT

#Gives Coordinates based on drone position and scan pixel
def calculate_world_coordinates(drone_pos, image_radius, img_x, img_y):
    pos_x = drone_pos[0] + (img_x-256)/512 * image_radius * 2
    pos_y = drone_pos[1] + (img_y-256)/512 * image_radius * 2
    return (pos_x, pos_y)


    return array_width,array_height
def getOverlapArray(waypoints,img_width=512,img_height=512):
    #TODO change MinX to value instead of array,its probably not needed 
    MinX = [0,float('inf')]
    MaxX = [0,float('-inf')]
    MinY = [0,float('inf')]
    MaxY = [0,float('-inf')]
    
    for i in range(len(waypoints)):
        x = waypoints[i][0]
        y = waypoints[i][1]
        #Determine the Minimum X,Y and Maximum X,Y of the sampling positions and saves which drone
        if (x < MinX[1]):
            MinX = [i, x]
        if (x > MaxX[1]):
            MaxX = [i, x]
        if (y < MinY[1]):
            MinY = [i, y]
        if (y > MaxY[1]):
            MaxY = [i, y]
    
    MinX[1] = calculate_world_coordinates(waypoints[MinX[0]], camera_offset, 0,0)[0]
    MaxX[1] = calculate_world_coordinates(waypoints[MaxX[0]], camera_offset, img_width,img_height)[0]
    MinY[1] = calculate_world_coordinates(waypoints[MinY[0]], camera_offset, 0,0)[1]
    MaxY[1] = calculate_world_coordinates(waypoints[MaxY[0]], camera_offset, img_width, img_height)[1]  
    
    #gets the needed width/height of the array based on the distance between smallest and largest X/Y Value, rounding up
    array_width = int(np.ceil(((MaxX[1]-MinX[1])*img_width)/(camera_offset*2)))
    array_height = int(np.ceil(((MaxY[1]-MinY[1])*img_height)/(camera_offset*2)))

    #fills the array with zeroes
    visibility_array = np.zeros((int(array_width), (int(array_height))))
    
    visibility_threshold= 3400 #once depth is higher than this value, the ground counts as seen
    
    #adds 1/NUM_DRONES if the point is seen
    for id in range(len(waypoints)):
        for i in range(img_width):
            for j in range(img_height):
                if swarm.depth_images[id][i][j][0] > visibility_threshold:
                    world_x, world_y = calculate_world_coordinates(waypoints[id], camera_offset, j, i)
                    world_x_idx = int(((world_x - MinX[1]) / (MaxX[1] - MinX[1])) * array_width)
                    world_y_idx = int(((world_y - MinY[1]) / (MaxY[1] - MinY[1])) * array_height)
                    if 0 <= world_x_idx < array_width and 0 <= world_y_idx < array_height:
                        visibility_array[world_x_idx][world_y_idx] += 1/NUM_DRONES
    
    
    #gets the corner of the visibility array, so correct overlap with probability array can be calculated

    visibility_offset_x=int(((MinX[1]+grid_offset)/GRID_SIZE)*prob_array_size)
    visibility_offset_y=int(((MinY[1]+grid_offset)/GRID_SIZE)*prob_array_size)
    
    visibility_offset=[visibility_offset_x,visibility_offset_y]
    return visibility_array, visibility_offset
    


In [5]:
def getProbabilityGrid(Last_Known_Position):
    #Create Target Probability Grid
    x = np.linspace(-grid_offset, grid_offset, prob_array_size)
    y = np.linspace(-grid_offset, grid_offset, prob_array_size)
    x, y = np.meshgrid(x, y)

    sigma= 10

    prob_density = (1 / (np.sqrt(2 * np.pi)* sigma)) * np.exp(-((x - Last_Known_Position[0])**2 + (y - Last_Known_Position[1])**2) / (2 * sigma**2))

    return prob_density,x,y

In [6]:
#return percentage of ground visible (depth>34meter)
def check_ground_visibility(depth_image):
    number_visible_ground=0
    for i in depth_image:
        for j in i:
            if j[0]>3400:
                number_visible_ground= number_visible_ground+1
    return number_visible_ground/(512**2)
 
def update_Target_Position():
    if Target_Position[0]<25:
        Target_Position = Target_Position + [0.5,0.5]

In [7]:
def mutate(solution):
    return null

# Waypints in Solution get randomized
def crossover(parent1, parent2):
    return null

Initialization of Drones and Solution Populations

In [8]:
#Spawn Drones and move to initial position

# Create the swarm object by passing the name
# of the world from the .sdf world file.
swarm = Swarm("rhea_swarm")

# Spawn X drones and keep the returning ids as handles
ids = swarm.spawn(NUM_DRONES)

#Create Fake Person Location
person_waypoint =np.array([5,5])

# First waypoints
waypoints = np.empty((0,3),float)
for i in range (NUM_DRONES):
    waypoints= np.append(waypoints,np.array([[math.ceil(i-(NUM_DRONES/2)),0.0,DRONE_HEIGHT]]), axis=0)

#create Initial
swarm.waypoints(ids, waypoints)

#Initialize Target Position
Target_Position = [10,10]

Last_Known_Position = Target_Position

#Visibility by Coordiantes File:
grid_visibility = np.full((AREA_SIZE_X,AREA_SIZE_Y), INITIAL_VISIBILITY)

#Create Target Probability Grid
sigma= 10

prob_density,x,y = getProbabilityGrid(Last_Known_Position)

#print(prob_density)


Subscribed successfully


In [9]:
#print(swarm.depth_images[1])
print(check_ground_visibility(swarm.depth_images[1]))


0.3119659423828125


In [10]:
# Problem classes
from pymoo.core.problem import Problem
from pymoo.optimize import minimize

# Algorithmss
from pymoo.algorithms.soo.nonconvex.de import DE

# Survival operators
from pymoo.operators.sampling.lhs import LHS

In [11]:
def scoreThatThing(prob_density,visibility_grid,visibility_offset):
        
    #scoring_array=np.array(prob_density)
    scoring_array=np.zeros(prob_density.shape)
    score=0
    
    #print("Visibility grid in scoring: ", visibility_grid.shape)
    
    for i in range(visibility_grid.shape[0]):
        for j in range(visibility_grid.shape[1]):
            #
            #scoring_array[i+visibility_offset[0]][j+visibility_offset[1]]*=visibility_grid[i][j]
            scoring_array[i+visibility_offset[0]][j+visibility_offset[1]]=prob_density[i+visibility_offset[0]][j+visibility_offset[1]]*visibility_grid[i][j]
            #only add score if half the drones see square
            if visibility_grid[i][j]>=0.5:
                score+=scoring_array[i+visibility_offset[0]][j+visibility_offset[1]]
                
    #plt.imshow(scoring_array, cmap='gray')
    #plt.show()
    return score

In [12]:
class MyProblem(Problem):

    def __init__(self, grid_size, n_drones, n_steps,waypoints,prob_density):
        
        self.grid_size = grid_size
        self.n_drones = n_drones
        n_var=2*n_drones
        self.n_steps = n_steps
        
        self.waypoints = waypoints
        self.prob_density = prob_density
        
        super().__init__(
            n_var=n_var,  # Variables in the solutions (x,y)
            n_obj=1,      # Number of Objectives we wanna optimize
            n_constr=0,   # No additional constraints, TODO: add no crashing into eachother
            xl=0,      # Lower boundaries of solution variables
            xu=1,       # Upper bounds of solution vars
        )    

        #x are the waypoints
    def _evaluate(self, x, out, *args, **kwargs):
        print("Starting eval")
        #scales the variables to  [-2,2]
        _x= np.array(x)*4-2
        
        #reshapes the variables back into a waypoint array
        coordinates = _x.reshape(-1, 2)        
        print(f"Reshaped coordinates: {coordinates.shape}")
        
        #send drones to solution it wants to score
        tempWaypoints = waypoints + np.pad(coordinates,((0, 0), (0, 1)),mode='constant',constant_values=0)
        swarm.waypoints(ids,tempWaypoints)
        
        isScored = False
        score = None
        
        while not isScored:
            #wait until all drones recieved
            if swarm.received_frames[ids[-1]]:
                print("All Frames recieved")
                visibility_grid,visibility_offset = getOverlapArray(tempWaypoints)
                score = scoreThatThing(self.prob_density,visibility_grid,visibility_offset)
                isScored=True
        
        print("Score is:", -score, " for solution \n",tempWaypoints)
        out["F"] = -score

In [13]:
from pymoo.core.sampling import Sampling
class MySampling(Sampling):

    def _do(self, problem, n_samples, **kwargs):
        X = np.random.rand(problem.n_var)
        print(f"Sampled X: {X.shape} -> {X}") 
        return X
    
from pymoo.termination import get_termination
termination = get_termination("n_gen", 10)

In [14]:
problem = MyProblem(GRID_SIZE, NUM_DRONES, NUM_GENERATIONS, waypoints, prob_density)

algorithm = DE(
    pop_size=50,
    sampling=MySampling(),    #Für initiale Population
    variant="DE/rand/1/bin",
    CR=0.9,
    F=0.8,
    dither="vector"
    #wieviel schritte in zukunft
)

In [15]:
finished=False
#Drone waypoints got from -25 to 25, Array from 0 to 50
#waypoints=np.array([[-5,0,35.],[-2,0,35.],[ 0,0,35.],[ 2,0,35.],[5,15,35.]])
#swarm.waypoints(ids,waypoints)

visibility_grid= "Null"

while not finished:
    #wait until all drones recieved
    if swarm.received_frames[ids[-1]]:
        
        #get best solution
        res = minimize(problem, algorithm, termination, seed=1, verbose=True)
        best_solution = res.X.reshape(-1, 2)
        print("Best Solution:",best_solution)
        #update waypoints
        waypoints = waypoints + np.pad(best_solution,((0, 0), (0, 1)),mode='constant',constant_values=0)
        swarm.waypoints(ids,waypoints)
        
        #update probability_grid and waypoints
        problem.waypoints = waypoints
        problem.prob_density = getProbabilityGrid(Last_Known_Position)
        
        #problem = MyProblem(GRID_SIZE, NUM_DRONES, NUM_GENERATIONS, self.waypoints, self.prob_density)
        

Sampled X: (10,) -> [4.17022005e-01 7.20324493e-01 1.14374817e-04 3.02332573e-01
 1.46755891e-01 9.23385948e-02 1.86260211e-01 3.45560727e-01
 3.96767474e-01 5.38816734e-01]
Starting eval
Reshaped coordinates: (5, 2)
All Frames recieved
Score is: -1310.457665210565  for solution 
 [[-2.33191198  0.88129797 35.        ]
 [-2.9995425  -0.79066971 35.        ]
 [-1.41297644 -1.63064562 35.        ]
 [-0.25495915 -0.61775709 35.        ]
 [ 1.5870699   0.15526694 35.        ]]


Exception: Population Set Attribute Error: Number of values (1): [-1310.45766521] and population size (10) do not match!

In [ ]:
#OLD ONE


finished=False
#Drone waypoints got from -25 to 25, Array from 0 to 50
#waypoints=np.array([[-5,0,35.],[-2,0,35.],[ 0,0,35.],[ 2,0,35.],[5,15,35.]])
#swarm.waypoints(ids,waypoints)

visibility_grid= "Null"

while not finished:
    #wait until all drones recieved
    if swarm.received_frames[ids[-1]]:
        visibility_grid,visibility_offset = getOverlapArray(waypoints)        
        
        import matplotlib.pyplot as plt 
        plt.imshow(visibility_grid, cmap='gray') 
        plt.show()
                

        #Find new best waypoints
        #best_solution = result.X
        #best_fitness = result.F
        #print("Best Solution: {} \n with a Score of {}".format(best_solution,best_fitness))
        
        #Move Drones to best waypoints
        #current_waypoints = best_solution
        #swarm.waypoints(ids, current_waypoints)
        
        #Update Target Model:
        sigma=sigma+0.5
        prob_density,x,y = getProbabilityGrid(Last_Known_Position)
        
        print("offset", visibility_offset)
        print("visibility:",visibility_grid.shape)        
        print("probability:",prob_density.shape)
        
        #Move Target-Person
        #update_Target_Position()
        #print("Moving Target to {}".format(Target_Position))
        plt.imshow(prob_density, cmap='gray')
        plt.show()
        
        whatever = scoreThatThing(prob_density,visibility_grid,visibility_offset)
        
        print("Score:",whatever)
        
        finished=True
        
        #if StopCondition:
            #finished=True